# Name : Manish Jayswal
# SR No.: 18844
# Degree : M.Tech Coursework(CDS)
# Subject : Numerical Optimization Assignment#3

In [138]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import hilbert #library to generate hilbert matrix
from sklearn.metrics import mean_squared_error

In [139]:
tolerance = 10**(-6)
dimensions = [5, 8, 12, 20]

# Q1(a)

In [140]:
def ConjugateGradient(n):
    H = hilbert(n)
    b = np.ones((n,1))
    x0 = np.zeros((n,1))
    r0 = H @ x0 - b
    p0 = -r0
    for i in range(200):
        s = H @ p0
        alpha = (r0.T @ r0) / (p0.T @ s)
        x_i = x0 + alpha * p0
        r_i = r0 + alpha * s
        beta = (r_i.T @ r_i) / (r0.T @ r0)
        p_i = -r_i + beta * p0
        norm = np.linalg.norm(r_i,2)
        if (norm<tolerance):
            break
            
        p0 = p_i
        x0 = x_i
        r0 = r_i
    return i
for dim in dimensions:
        print("The number of iterations to solve for dimension n = {} is {}".format(dim, ConjugateGradient(dim)))    

The number of iterations to solve for dimension n = 5 is 5
The number of iterations to solve for dimension n = 8 is 17
The number of iterations to solve for dimension n = 12 is 35
The number of iterations to solve for dimension n = 20 is 129


# Q1(b)

In [141]:
# Using SSOR preconditioner
def precod(n):
    H = hilbert(n)
    D = np.diag(np.diag(H)) #diagonal matrix
    L = np.tril(H)  #Strictly Lower Triangular matrix
    U = np.triu(H) #Strictly Upper Triangular matrix
    D_inv = np.linalg.inv(D)
    M = (D+L) @ D_inv @ (D+U)
    return M
    
#Preconditioned Conjugate Gradients Method    
def PreConCG(n):
    H = hilbert(n)
    b = np.ones((n,1))
    x0 = np.zeros((n,1))
    r0 = H @ x0 - b
    M = precod(n)
    M_inv = np.linalg.inv(M)
    A = np.linalg.cond(H)
    B = np.linalg.cond(M_inv @ H)
    p0 = - M_inv @ r0
    for i in range(200):
        s = H @ p0
        alpha = (r0.T @ M_inv @r0) / (p0.T @ s)
        x_i = x0 + alpha * p0
        r_i = r0 + alpha * s
        beta = (r_i.T @ M_inv @ r_i) / (r0.T @ M_inv @ r0)
        p_i = - M_inv @ r_i + beta * p0
        norm = np.linalg.norm(r_i, 2)
        if (norm<tolerance):
            break
            
        p0 = p_i
        x0 = x_i
        r0 = r_i
    return i

print("Using Preconditioned Conjugate Gradient:\n")
for dim in dimensions:
     print("The number of iterations to solve for dimension n = {} is {}".format(dim, PreConCG(dim)))    

Using Preconditioned Conjugate Gradient:

The number of iterations to solve for dimension n = 5 is 4
The number of iterations to solve for dimension n = 8 is 15
The number of iterations to solve for dimension n = 12 is 30
The number of iterations to solve for dimension n = 20 is 106


# Q1(c)

In [142]:
def Clustered(A,n):
    H = A @ A.T
    D = np.diag(np.diag(H)) #diagonal matrix
    L = np.tril(H)  #Strictly Lower Triangular matrix
    U = np.triu(H) #Strictly Upper Triangular matrix
    D_inv = np.linalg.inv(D)
    M = (D+L) @ D_inv @ (D+U)
    return M
    
def ConjugateGradient1(str1, n):
    A = np.random.rand(n,n)
    if(str1 =="Clustered"):
        H = Clustered(A,n)
    elif(str1=='nonClustered'):
        H = A @ A.T
    b = np.ones((n,1))
    x0 = np.zeros((n,1))
    r0 = H @ x0 - b
    p0 = -r0
    for i in range(1000):
        s = H @ p0
        alpha = (r0.T @ r0) / (p0.T @ s)
        x_i = x0 + alpha * p0
        r_i = r0 + alpha * s
        beta = (r_i.T @ r_i) / (r0.T @ r0)
        p_i = -r_i + beta * p0
        norm = np.linalg.norm(r_i,2)
        if (norm<tolerance):
            break
            
        p0 = p_i
        x0 = x_i
        r0 = r_i
    return i
distributions = ["Clustered", "nonClustered"]
for dim in [7,9,11,15,25]:
    print("\nThe number of iterations to solve for dimension n = {}:".format(dim))
    for distribution in distributions:
        print("\t\t\t\t\t\t using {} distribution is {}".format(distribution, ConjugateGradient1(distribution,dim)))    


The number of iterations to solve for dimension n = 7:
						 using Clustered distribution is 6
						 using nonClustered distribution is 7

The number of iterations to solve for dimension n = 9:
						 using Clustered distribution is 8
						 using nonClustered distribution is 9

The number of iterations to solve for dimension n = 11:
						 using Clustered distribution is 8
						 using nonClustered distribution is 12

The number of iterations to solve for dimension n = 15:
						 using Clustered distribution is 11
						 using nonClustered distribution is 17

The number of iterations to solve for dimension n = 25:
						 using Clustered distribution is 11
						 using nonClustered distribution is 31


# Q1(d)

In [143]:
X = np.load('data_x.npy')
mean = np.mean(X)
std = np.std(X)
y = np.exp(-(X-mean)**2/(2*std**2)) / (std*(2*np.pi)**(1/2))
order = 8  #optimal order 8
A1 = np.zeros((len(y), order))
for i in range(len(y)):
    for j in range(order):
        A1[i][j] = X[i]**j

H = A1.T @ A1
b = np.array([A1.T @ y])

In [144]:
def ConjugateGradient2(n):
    x0 = np.zeros((n,1))
    r0 = H @ x0 - b.T
    p0 = -r0
    xlist1 = []
    for i in range(500):
        xlist1.append(x0)
        s = H @ p0
        alpha = (r0.T @ r0) / (p0.T @ s)
        x_i = x0 + alpha * p0
        r_i = r0 + alpha * s
        beta = (r_i.T @ r_i) / (r0.T @ r0)
        p_i = -r_i + beta * p0
        norm = np.linalg.norm(r_i,2)
        if (norm<tolerance):
            break
            
        p0 = p_i
        x0 = x_i
        r0 = r_i
    return xlist1


In [145]:
xlist1 = ConjugateGradient2(len(H))
b_calculated = xlist1[-1]
y_predicted = A1 @ b_calculated
MSE = mean_squared_error(Y, y_predicted)

print("\nThe coefficient vector for this model is : \n{}".format(b_calculated))
print("\nMSE : {}".format(MSE))


The coefficient vector for this model is : 
[[-3.15185383e+00]
 [ 3.56593585e-01]
 [-1.55715673e-02]
 [ 3.17139347e-04]
 [-2.53969826e-06]
 [-7.99050722e-09]
 [ 2.44966532e-10]
 [-1.05359044e-12]]

MSE : 1.1567876633495509e-07


# Q(2)

In [146]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from numpy.linalg import norm
from math import sqrt
tol = 10**(-5)

In [147]:
#Rosenbrock's function
def f(x):
    return (1-x[0])**2 + 20*(x[1]-x[0]**2)**2

def J(x):
    dfx = 2*(x[0]-1) -4*20*x[0]*(x[1]-x[0]**2)
    dfy = 2*20*(x[1]-x[0]**2)
    return np.array([dfx, dfy])
    
def H(x):
    dfxx = 2 - 4*20*(x[1]-3*x[0]**2) 
    dfxy = dfyx = -4*20*x[0]
    dfyy = 2*20
    return np.array([[dfxx, dfxy],[dfyx, dfyy]])

In [148]:
def dogleg_subProblem(Bk, gk, Hk, rk):
    Hk = np.linalg.inv(Bk)
    pU = - ((gk.T @ gk) / (gk.T @ Bk @ gk)) * gk
    if norm(pU) >= rk:
        return rk * pU / norm(pU)
    pB = -Hk @ gk
    if norm(pB) <= rk:
        return pB
    pB_pU = pB - pU
    Tau = (-(pU.T @ pB_pU) + sqrt((pU.T @ pB_pU)**2 - pB_pU.T @ pB_pU  * (pU.T @ pU - rk**2))) / (pB_pU.T @ pB_pU)
    result = pU + pB_pU * Tau
    return result

In [149]:
def trustRegion(xk,rk):
    k = 0
    xlist = [xk]
    itr_list = [k]    
    maxm_iter = 100
    while (1):
        gk = J(xk)
        Bk = H(xk)
        Hk = np.linalg.inv(Bk)
        
        pk = dogleg_subProblem(Bk, gk, Hk, rk)
        numerator = f(xk) - f(xk + pk)
        denominator = -(gk.T @ pk + 0.5 * pk.T @ Bk @ pk)
        if denominator == 0.0:
            rhok = 1e100
        else:
            rhok = numerator / denominator
        norm_pk = norm(pk)
        if rhok < rho_l:
            rk = 0.25 * norm_pk
        else: 
            if rhok > rho_h and norm_pk == rk:
                rk = min(2.0*rk, rm)
            else:
                rk = rk
        if rhok > eta:
            xk = xk + pk
        else:
            xk = xk
        if norm(gk) < tol:
            break
        
        if k >= maxm_iter:
            break
        k = k + 1
        xlist.append(xk)
        itr_list.append(k)
    return itr_list, xlist    

In [150]:
r = 1.0
rm = 100.0
eta = 0.2

rho_l = 0.25
rho_h = 0.75

# Initial guesses 
x = np.array([[0,-1],[0,0.5]])
for i in range(2):
    num_iteration, x_list = trustRegion(x[i],r)
    print("\nFor initial guess: x0 = {}, the total number of iterations taken are {}".format(x[i],num_iteration[-1]))
    print("Computed solution of of given problem using dogleg method is x = {}".format(x_list[-1]))
    print("The value of function at this point is {}".format(f(x_list[-1])))


For initial guess: x0 = [ 0. -1.], the total number of iterations taken are 11
Computed solution of of given problem using dogleg method is x = [0.99999996 0.99999991]
The value of function at this point is 2.9477808753850633e-15

For initial guess: x0 = [0.  0.5], the total number of iterations taken are 15
Computed solution of of given problem using dogleg method is x = [0.99999987 0.99999973]
The value of function at this point is 1.8247415784723367e-14
